In [1]:
import sqlite3
import pandas as pd
import plotly.express as px
import sys
import plotly.graph_objects as go

sys.path.append('../')
from functions_env import DB_PATH


conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

In [2]:
df_deathcity = pd.read_csv('/Users/charlesdedampierre/Desktop/cultura_project/raw_data/wikidata_data/deathcity_location.csv')
df_deathcity = df_deathcity[['deathcity_wiki_id', 'location']]
df_deathcity = df_deathcity.dropna()
df_deathcity  = df_deathcity[df_deathcity['location'].str.contains('Point')]
df_deathcity['city_longitude'] = df_deathcity['location'].apply(lambda x: x.split(' ')[0].split('(')[1]).astype(float)
df_deathcity['city_latitude'] = df_deathcity['location'].apply(lambda x: x.split(' ')[1].split(')')[0]).astype(float)
df_deathcity = df_deathcity.drop('location', axis=1)
df_deathcity = df_deathcity.rename(columns={'deathcity_wiki_id':'deathcity_wikidata_id'})


In [3]:
df_ind_deathcity = pd.read_sql_query("SELECT * FROM individual_deathcity", conn)
df_ind_regions = pd.read_sql_query("SELECT * FROM individuals_regions", conn)
df_fin_deathcity = pd.merge(df_deathcity, df_ind_deathcity, on = 'deathcity_wikidata_id')


df_year = pd.read_sql_query("SELECT * FROM individuals_main_information", conn)
df_year['productive_year'] = df_year['birthyear'] + 35
df_year = df_year[['individual_wikidata_id', 'productive_year']]
df_year = df_year.dropna()
df_year['decade'] = df_year['productive_year'].apply(lambda x: round(x / 10) * 10)


In [30]:
from ipyleaflet import Map, Heatmap

In [42]:

re_region = 'Chinese world'
min_date= 1400
max_date = 1850
df_region = pd.merge(df_ind_regions, df_year)

df_map =df_region[df_region['region_name']==re_region]
df_map = df_map[(df_map['decade']>=min_date)&(df_map['decade']<=max_date)]

df_map = pd.merge(df_map, df_fin_deathcity, on = 'individual_wikidata_id')
data_loc = df_map.groupby(['city_latitude', 'city_longitude'])['individual_wikidata_id'].count().reset_index()

latitudes = data_loc['city_latitude'].values
longitudes = data_loc['city_longitude'].values
count = data_loc['individual_wikidata_id'].values

min_opacity = 0.7
blur = 12
radius = 10
zoom = 4

m = Map(center= data_loc[['city_latitude', 'city_longitude']].mean().to_list(), zoom=zoom)
#m = Map(zoom=3)

heatmap = Heatmap(locations=[[latitudes[i], longitudes[i], count[i]] for i in range(len(data_loc))],
                  radius=radius, min_opacity=min_opacity, blur=blur)

m.add_layer(heatmap)
m

Map(center=[31.182779419125, 112.9747476495], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom…

AttributeError: 'Map' object has no attribute 'html'

In [ ]:
import sys
from tqdm import tqdm

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np

import itertools

sys.path.append('../')

from sys_utils import load_model
from data_model_region import Region
from data_model import Individual
import typing as t
import random
from tqdm import tqdm

In [ ]:
checkpoint_path = '../checkpoints_dev'

regions = load_model(
        Region, name=checkpoint_path+"/regions.jsonl"
    )

individuals = load_model(
        Individual, name=checkpoint_path + "/individuals.jsonl"
    )

In [ ]:
individuals_filtered = [x for x in individuals if x.regions != None]
individuals_filtered = [x for x in individuals_filtered if x.id.raw_birthcities != None]


In [ ]:
df_individuals = [
    {
        "wikidata_id": x.id.wikidata_id,
        "identifier":[y.location for y in x.id.raw_birthcities],
        "region_code": x.regions,
         "range_impact": x.impact_years,
        'cultural_score': x.cultural_score
    }
    for x in individuals_filtered
]

df_individuals = pd.DataFrame(df_individuals)

In [ ]:
df_region = df_individuals.explode('region_code')
